In [1]:
%run qsu.ipynb
import numpy as np
import sympy
import networkx as nx
from qecsim import paulitools as pt 
from qecsim.models.generic import DepolarizingErrorModel
from qecsim.models.toric import ToricCode
import nbimporter
import galois
from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
from typing import List, Tuple
import hypergraph_prod_code as hpc

In [2]:
def belief_prop(H: np.array, s: np.array, p: float, max_iter: int) -> Tuple:
    """ 
    Belief Propagation Algorithm for Decoding LDPC Codes

    Parameters:
    -----------
    H - parity-check matrix corresponding to either X or Z checks
    s - Error syndrome
    p - Channel error rate for chosen noise channel
    max_iter - Maximum number of iterations to run BP algorithm for
    """
    data_to_parity = np.zeros((len(H[0]),len(H)), dtype=float)
    parity_to_data = np.zeros((len(H), len(H[0])), dtype=float)
    H_tanner_graph = hpc.parity_check_mat_to_tanner(H)
    
    # Channel Log Likelihood Ratio
    p_l = np.log((1 - p)/p)
    
    P_1 = np.zeros((len(H[0]),), dtype=float)
    e_BP = np.zeros((len(H[0]),), dtype=float)

    # (1) Initialization
    for edge in H_tanner_graph.edges:
        data_node_num = int(edge[0][1:])
        parity_node_num = int(edge[1][1:])
        data_to_parity[data_node_num][parity_node_num] = p_l 

    for iter in range(1, max_iter + 1):
        # Scaling Factor
        a = 1 - 2**(-1 * iter)

        # (2) Parity to Data Messages
        for edge in H_tanner_graph.edges:
            parity_node_num = int(edge[1][1:])
            data_node_num = int(edge[0][1:])

            # Get list of neighbors of current parity_node set minus the current data node
            V = list(nx.neighbors(H_tanner_graph, edge[1]))
            V.remove(edge[0])

            # Get messages from elements of V to current parity node
            data_to_par_msgs = [data_to_parity[int(v[1:])][parity_node_num] for v in V]
            w = np.min([np.abs(msg) for msg in data_to_par_msgs])
            parity_to_data[parity_node_num][data_node_num] = ((-1) ** int(s[parity_node_num])) * a * np.prod(np.sign(data_to_par_msgs)) * w 

        # (3) Data to Parity Messages
        for edge in H_tanner_graph.edges:
            data_node_num = int(edge[0][1:])
            parity_node_num = int(edge[1][1:])

            # Get list of neighbors of current data node set minus the current parity node
            U = list(nx.neighbors(H_tanner_graph, edge[0]))
            U.remove(edge[1])

            # Get messages from elements of U to current data node
            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            data_to_parity[data_node_num][parity_node_num] = p_l + np.sum(par_to_data_msgs)

        # Hard Decision
        for edge in H_tanner_graph.edges:
            data_node_num = int(edge[0][1:])
            parity_node_num = int(edge[1][1:])

            # Get list of neighbors of current data node
            U = list(nx.neighbors(H_tanner_graph, edge[0]))

            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            P_1[data_node_num] = p_l + np.sum(par_to_data_msgs)
            e_BP[data_node_num] = -1 * np.sign(P_1[data_node_num])
        
        # (4) Termination Check
        e_BP = e_BP * (e_BP > 0)
        #print(e_BP)
        if (np.array_equal(np.dot(H, e_BP), s)):
            return True, e_BP, P_1 

    return False, e_BP, P_1

In [3]:
def OSD_0(H: np.array, P_1: np.array, s: np.array) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - parity check matrix
    P_1 - BP soft decision vector
    s - Error syndrome 

    Returns:
    --------
    Error string
    """
    GF = galois.GF(2)

    # Get rank of parity check matrix 
    H_rank = np.linalg.matrix_rank(GF(H))
    
    # Sort soft decision vector 
    P_1_sorted_pos = np.argsort(P_1, kind='stable')
    P_1_sorted = [P_1[i] for i in P_1_sorted_pos]

    # Remove remove row from 'H' and related position from 's'
    del_pos = np.random.choice(len(H))
    H_d = np.delete(H, del_pos, axis=0)
    s_d = GF(np.delete(s, del_pos, axis=0))

    # Rearrange columns of H to match the reordered soft-decision vector
    H_d[:] = H_d[:, P_1_sorted_pos]

    # Select first RANK(H) linearly independent columns of above rearrangement
    H_rref, inds = sympy.Matrix(H_d).rref()
    H_rref = np.array(H_rref)
    H_S = np.vstack(([H_d[:, inds[i]] for i in range(0, H_rank)])).T
    H_S_inv = GF(np.mod((np.linalg.inv(H_S)).astype('int32'),2))
    
    # Calculate the OSD-0 solution on the basis-bits
    e_S = H_S_inv @ s_d
    e_ST = np.hstack((e_S, GF(np.zeros((len(H[0]) - H_rank,), dtype='int32'))))
    
    
    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = GF(np.zeros((len(H[0]),), dtype='int32'))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]
    
    return e_OSD


In [4]:
def mod(x,modulus):
    numer, denom = x.as_numer_denom()
    return numer*sympy.mod_inverse(denom,modulus) % modulus    

# Turn to Ordered Statistics Decoding if BP fails to converge
def OSD_0_V1(H: np.array, P_1: np.array, s: np.array) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - parity check matrix
    P_1 - BP soft decision vector
    s - Error syndrome 

    Returns:
    --------
    Error string
    """
    GF = galois.GF(2)
    # Get the rank of the parity check matrix
    H_rank = np.linalg.matrix_rank(GF(H))
    print(H_rank)

    # Maintain a mapping between bit positions and elements of the BP soft-decision vector
    P_1_sorted_pos = np.argsort(P_1, kind='stable')[::-1]
    P_1_sorted = [P_1[i] for i in P_1_sorted_pos]
    print(P_1_sorted_pos)
    print(P_1_sorted)

    # Rearrange columns of H to match the reordered soft-decision vector
    H[:] = H[:, P_1_sorted_pos]

    # Randomly remove a row from 'H' and related position from syndrome
    del_pos = np.random.choice(len(H))
    print(del_pos)
    print(H,s)
    H_del = np.delete(H, del_pos, axis=0)
    s_del = np.delete(s, del_pos)

    # Select first RANK(H) linearly independent columns of above rearrangement
    H_rref, inds = sympy.Matrix(H_del).rref()

    H_rref = H_rref.applyfunc(lambda x : mod(x,2))
    H_rref, inds = sympy.Matrix(H_rref).rref()
    H_S = np.vstack(([H_del[:, inds[i]] for i in range(0, H_rank)]))
    print(H_S)
    H_S_inv = np.mod(np.linalg.inv(H_S), 2)
    print(H_S_inv)
    

    # Calculate the OSD-0 solution on the basis-bits
    e_S = np.linalg.inv(H_S) @ s_del
    e_ST = np.hstack((e_S, np.zeros((len(H[0]) - H_rank,))))

    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = np.zeros((len(H[0]),))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]

    return e_OSD

In [5]:
def get_H_J(H:np.array, J: List, s_p: np.array):
    if (J == []):
        H_J = []
        H_J_s = np.array(s_p.T)
    else:
        H_J = np.vstack([H[:, i] for i in J])
        H_J_s = np.vstack((H_J,s_p)).T
    return H_J, H_J_s

def replace_pos(e_p: np.array, x: np.array, J: List):
    count = 0
    for i in range(len(e_p)):
        if(i in J):
            e_p[i] = x[count]
            count += 1

    return e_p


# Turn to Ordered Statistics Decoding if BP fails to converge
def OSD_0_V2(H: np.array, s: np.array, P_1: np.array, e_p: np.array, num_qubits: int) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - Parity check matrix
    s - Error syndrome 
    P_1 - Soft decision vector
    e_p - Hard decision vector
    num_qubits - Number of qubits

    Returns:
    --------
    Error string e such that He = s
    """

    """
    # Maintain a mapping between bit positions and elements of the BP soft-decision vector
    P_1_sorted_pos = np.argsort(P_1, kind='stable')[::-1]
    P_1_sorted = [P_1[i] for i in P_1_sorted_pos]

    # Rearrange columns of H to match the reordered soft-decision vector
    H[:] = H[:, P_1_sorted_pos]
    """

    # Want to construct information set J
    J = []
    s_p = np.mod(s + (H @ e_p),2)
    for i in range(num_qubits):
        H_J, H_J_s = get_H_J(H, J, s_p)
        if (np.linalg.matrix_rank(H_J_s) == np.linalg.matrix_rank(H_J)):
            break 
        J_p = J + [i]
        H_J_p, _ = get_H_J(H, J_p, s_p)
        if (np.linalg.matrix_rank(H_J_p) > np.linalg.matrix_rank(H_J)):
            J = J_p
            s_p += e_p[i] * H[:,i]
    rref, _ = sympy.Matrix.rref(sympy.Matrix(H_J_s))
    x = rref[:,-1]
    e = replace_pos(e_p, x, s_p)
    return e

In [6]:
def OSD_0_V3(H: np.array, s: np.array, P_1: np.array, e_p: np.array, num_qubits: int) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - Parity check matrix
    s - Error syndrome 
    P_1 - Soft decision vector
    e_p - Hard decision vector
    num_qubits - Number of qubits

    Returns:
    --------
    Error string e such that He = s
    """
    # Maintain a mapping between bit positions and elements of the BP soft-decision vector
    P_1_sorted_pos = np.argsort(P_1, kind='stable')[::-1]
    P_1_sorted = [P_1[i] for i in P_1_sorted_pos]

    # Rearrange columns of H to match the reordered soft-decision vector
    H[:] = H[:, P_1_sorted_pos]

    # Solve for He=s
    s = np.reshape(s, (len(s),1))
    aug_mat = sympy.Matrix(np.concatenate((H,s), axis=1), domain=sympy.ZZ(2))
    aug_mat, inds = sympy.Matrix.rref(aug_mat)
    print(np.array(aug_mat))
    
    

In [12]:
# Loop to see probability distribution evolution across many runs of BP decoder
def prob_likelihood_evol(dim: int):
    # Array of likelihoods (soft-decision vectors) for each run
    BP_like_Z = []
    BP_like_X = []

    # Define code
    my_code = ToricCode(dim, dim);
    my_error_model = DepolarizingErrorModel()
    
    # Define error probability and randomly select seed for errors
    error_probability = 0.1 
    rn = np.random.randint(1,100000)
    rng = np.random.default_rng(rn)

    # Get random error
    error = my_error_model.generate(my_code, error_probability, rng)

    # Get syndrome of error
    syndrome = pt.bsp(error, my_code.stabilizers.T)

    # Get X and Z parity check matrices and related error vectors 
    Z_stabs = (my_code.stabilizers[:dim ** 2])[:, 2 * dim ** 2:]
    X_stabs = (my_code.stabilizers[dim ** 2:])[:, :2 * dim ** 2]
    Z_error = error[2 * dim**2:]
    X_error = error[:2 * dim**2]
    Z_syndrome = np.mod(X_stabs @ Z_error,2)
    X_syndrome = np.mod(Z_stabs @ X_error, 2)

    for num_iter in range(1, 2 * dim ** 2 + 1):
        converged_Z, e_BP_Z, P_1_Z = belief_prop(X_stabs, Z_syndrome, error_probability, num_iter)
        converged_X, e_BP_X, P_1_X = belief_prop(Z_stabs, X_syndrome, error_probability, num_iter)
        
        BP_like_Z.append(P_1_Z)
        BP_like_Z.append(P_1_X)

    BP_like_Z = np.array(BP_like_Z)
    BP_like_X = np.array(BP_like_X)
    return (BP_like_Z, BP_like_X)

In [26]:
import pandas as pd

def f(x, y, data):
    for i in x:
        for j in y:
            print(i,j)
    return result

dim = 3
SD_Z, SD_X = prob_likelihood_evol(dim)
x = np.linspace(1, 2 * dim ** 2, 4 * dim ** 2)
y = np.linspace(1, 2 * dim ** 2, 2 * dim ** 2)
X, Y = np.meshgrid(x,y)
Z_data_Z = f(X,Y,np.array(SD_Z, dtype=int))
X_data_Z = f(X,Y,np.array(SD_X, dtype=int))

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z_data_Z, 50, cmap='binary')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');



[ 1.          1.48571429  1.97142857  2.45714286  2.94285714  3.42857143
  3.91428571  4.4         4.88571429  5.37142857  5.85714286  6.34285714
  6.82857143  7.31428571  7.8         8.28571429  8.77142857  9.25714286
  9.74285714 10.22857143 10.71428571 11.2        11.68571429 12.17142857
 12.65714286 13.14285714 13.62857143 14.11428571 14.6        15.08571429
 15.57142857 16.05714286 16.54285714 17.02857143 17.51428571 18.        ] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.          1.48571429  1.97142857  2.45714286  2.94285714  3.42857143
  3.91428571  4.4         4.88571429  5.37142857  5.85714286  6.34285714
  6.82857143  7.31428571  7.8         8.28571429  8.77142857  9.25714286
  9.74285714 10.22857143 10.71428571 11.2        11.68571429 12.17142857
 12.65714286 13.14285714 13.62857143 14.11428571 14.6        15.08571429
 15.57142857 16.05714286 16.54285714 17.02857143 17.51428571 18.        ] [2. 2. 2. 2

NameError: name 'result' is not defined

In [7]:
# initialize models
dim = 3
my_code = ToricCode(dim,dim)
my_error_model = DepolarizingErrorModel()
GF = galois.GF(2)

In [27]:
# Set physical error probability to 10%
error_probability = 0.1
# Seed random number generator for repeatability
#rng = np.random.default_rng(15355)
rng = np.random.default_rng()

# Error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
print(error)
print(('error:\n{}'.format(my_code.new_pauli(error))))

[0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
error:
┼─·─┼─Z─┼─Y
·   X   ·  
┼─·─┼─·─┼─·
·   Y   ·  
┼─·─┼─·─┼─·
·   ·   ·  


In [28]:
# Syndrome: Stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
print(syndrome)
print(('syndrome:\n{}'.format(my_code.ascii_art(syndrome))))

[1 1 1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0]
syndrome:
X───X───┼──
│ Z │ Z │ Z
┼───X───┼──
│ Z │ Z │  
┼───X───┼──
│   │   │ Z


In [29]:
Z_stabs = (my_code.stabilizers[:dim ** 2])[:, 2 * dim ** 2:]
X_stabs = (my_code.stabilizers[dim ** 2:])[:, :2 * dim ** 2]
#print(str(Z_stabs) + "\n" + str(X_stabs))
#print(np.mod(X_stabs @ Z_stabs.T,2))
Z_error = error[2 * dim**2:]
X_error = error[:2 * dim**2]
print("Z_error: " + str(Z_error))
print("X_error: " + str(X_error))

Z_error: [0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
X_error: [0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0]


In [30]:
Z_syndrome = np.mod(X_stabs @ Z_error,2)
print("Z_syndrome: " + str(Z_syndrome))
X_syndrome = np.mod(Z_stabs @ X_error, 2)
print("X_syndrome: " + str(X_syndrome))

Z_syndrome: [0 1 0 0 1 0 1 1 0]
X_syndrome: [1 1 1 1 1 0 0 0 1]


In [31]:
converged_Z, e_BP_Z, P_1_Z = belief_prop(X_stabs, Z_syndrome, 0.1, 2 * dim ** 2)
converged_X, e_BP_X, P_1_X = belief_prop(Z_stabs, X_syndrome, 0.1, 2 * dim ** 2)

[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0.  1.  1.  1. -0. -0. -0. -0. -0.  1. -0. -0.  1. -0. -0. -0. -0.]
[-0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0.  1.  1.  1. -0. -0. -0. -0. -0.  1. -0. -0.  1. -0. -0. -0. -0.]
[-0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0.  1.  1.  1. -0. -0. -0. -0. -0.  1. -0. -0.  1. -0. -0. -0. -0.]
[-0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0.  1.  1.  1. -0. -0. -0. -0. -0.  1. -0. -0.  1. -0. -0. -0. -0.]
[-0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0.  1.  1.  1. -0. -0. -0. -0. -

In [32]:
print(converged_Z, e_BP_Z, P_1_Z)
print('\n')
print(converged_X, e_BP_X, P_1_X)

True [ 1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0.] [-1.64791843  3.63915321  2.19722458  2.19722458  3.63915321  4.60043896
  2.19722458  3.63915321  4.60043896  2.19722458  1.23593882  4.60043896
  4.60043896 -0.2059898   4.60043896  2.19722458  1.23593882  4.60043896]


False [-0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.] [ 2.19722458  2.19722458 -2.19720781  0.2661409   0.2661409   4.12827473
  2.19722458  2.19722458  2.19722458  2.19719105  0.2661409   2.19719105
  2.19722458  0.2661409   2.19722458  2.19722458  4.12830826  2.19722458]


In [33]:
if (converged_X == True and converged_Z == True):
    e_OSD_Z = e_BP_Z
    e_OSD_X = e_BP_X
if (converged_Z == False):
    print("BP ON Z FAILED")
    #e_OSD_Z = np.mod(OSD_0_V2(X_stabs, Z_syndrome, P_1_Z, e_BP_Z, 2 * dim ** 2), 2)
    #e_OSD_Z = np.mod(OSD_0_V3(X_stabs, Z_syndrome, P_1_Z, e_BP_Z, 2 * dim ** 2), 2)
    e_OSD_Z = np.array(OSD_0(X_stabs, P_1_Z, Z_syndrome))
    if (converged_X == True):
        e_OSD_X = np.zeros((len(e_OSD_Z),))
    print(e_OSD_Z)
if (converged_X == False):
    print("BP ON X FAILED")
    #e_OSD_X = np.mod(OSD_0_V2(Z_stabs, X_syndrome, P_1_X, e_BP_X, 2 * dim ** 2), 2)
    #e_OSD_X = np.mod(OSD_0_V3(Z_stabs, X_syndrome, P_1_X, e_BP_X, 2 * dim ** 2), 2)
    e_OSD_X = np.array(OSD_0(Z_stabs, P_1_X, X_syndrome))
    if (converged_Z == True):
        e_OSD_Z = np.zeros((len(e_OSD_X),))
    print(e_OSD_X)

BP ON X FAILED
[0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [34]:
# Check consistency with syndromes measured.
print(np.mod(X_stabs @ e_OSD_Z, 2))
print(np.mod(Z_stabs @ e_OSD_X, 2))

[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 1 0 0 0 1]


In [35]:
print("Error: " + str(error))
recovery = np.array((np.hstack((e_OSD_X, e_OSD_Z))), dtype='int32')
print("Recovery operation: " + str(recovery))

# check recovery ^ error commutes with stabilizers
print("Commutation with stabilizers: " + str(pt.bsp(recovery ^ error, my_code.stabilizers.T)))

# success iff recovery ^ error commutes with logicals
print("Commutation with logicals: " + str(pt.bsp(recovery ^ error, my_code.logicals.T)))

Error: [0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Recovery operation: [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Commutation with stabilizers: [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0]
Commutation with logicals: [1 1 0 0]
